<a href="https://colab.research.google.com/github/nicoelbert/vehicleroutingproblem/blob/main/vehicle_routing_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vehicle Routing Problem
vehicle routing problem on complete information using simmulated annealing

#Preparation

In [1]:
!git clone 'https://github.com/nicoelbert/vehicleroutingproblem'

Cloning into 'vehicleroutingproblem'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 62 (delta 28), reused 39 (delta 14), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [7]:
#import packages
import pandas as pd

import sys
#import classes


!git pull
%cd vehicleroutingproblem

from classes import classes as cl

Already up to date.
[Errno 2] No such file or directory: 'vehicleroutingproblem'
/content/vehicleroutingproblem


In [3]:
!git pull
!ls

Already up to date.
classes  original_data	README.md  vehicle_routing_solver.ipynb


##Data Import

In [4]:
#depot masterdata

#load Masterdata from csv
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/Depots.csv'
df_depots = pd.read_csv(url)


#load basetable
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/preparedJobs.csv'
df_jobs = pd.read_csv(url)


#next steps load data into classes
#create dictionary for each class

In [10]:
#backup

"Evaluation of Constructionsites"
print(len(df_jobs['plz'].unique()))
print(len(df_jobs['lon'].unique()))
print(len(df_jobs['lat'].unique()))
#print(len(df_jobs[['lon','lat']]()))

df_sites = df_jobs[['lon','lat']].copy()

print(df_sites.shape)

df_sites.drop_duplicates(inplace=True)


print(df_sites.shape)
k.key

6166
6158
6158
(53178, 2)
(6158, 2)


'701010'